# New Section

In [ ]:
!pip install flair

     |████████████████████████████████| 286kB 6.8MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 798kB 27.9MB/s 
     |████████████████████████████████| 776.8MB 24kB/s 
     |████████████████████████████████| 71kB 10.8MB/s 
     |████████████████████████████████| 2.1MB 52.7MB/s 
     |████████████████████████████████| 19.7MB 1.2MB/s 
     |████████████████████████████████| 983kB 48.9MB/s 
     |████████████████████████████████| 1.2MB 44.9MB/s 
     |████████████████████████████████| 3.3MB 45.6MB/s 
     |████████████████████████████████| 901kB 47.7MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=3fc5693110d48aa95efa146dcf64b1768c6f8a8dc4c4b6819ea07722444fcf44
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Created wheel for mpld3: filename

In [ ]:
!pip install allennlp==0.9.0

     |████████████████████████████████| 7.6MB 6.2MB/s 
     |████████████████████████████████| 5.6MB 32.0MB/s 
     |████████████████████████████████| 133kB 53.9MB/s 
     |████████████████████████████████| 163kB 59.4MB/s 
     |████████████████████████████████| 245kB 52.5MB/s 
     |████████████████████████████████| 133kB 60.3MB/s 
     |████████████████████████████████| 122kB 52.7MB/s 
     |████████████████████████████████| 30.8MB 112kB/s 
     |████████████████████████████████| 51kB 9.4MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 266kB 51.6MB/s 
     |████████████████████████████████| 256kB 55.6MB/s 
     |████████████████████████████████| 81kB 11.2MB/s 
     |████████████████████████████████| 7.5MB 37.1MB/s 
     |████████████████████████████████| 153kB 59.7MB/s 
     |████████████████████████████████| 3.2MB 51.8MB/s 
     |████████████████████████████████| 92kB 13.0MB/s 
     |████████████████████████████████| 2.1MB 49.8MB/s 

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

columns = {0:'text',1:'ner'}

data_folder = '/content/Data'

corpus:Corpus=ColumnCorpus(data_folder,columns,train_file='fi_en_train.txt',test_file='fi_en_test.txt')
corpus = corpus.downsample(0.8)
print(corpus)

2021-05-03 10:41:25,343 Reading data from /content/Data
2021-05-03 10:41:25,344 Train: /content/Data/fi_en_train.txt
2021-05-03 10:41:25,349 Dev: None
2021-05-03 10:41:25,351 Test: /content/Data/fi_en_test.txt
Corpus: 5128 train + 570 dev + 680 test sentences


In [ ]:
tag_type = 'ner'# make tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary)

Dictionary with 7 tags: <unk>, O, B-location, I-location, , <START>, <STOP>


In [ ]:
from flair.embeddings import WordEmbeddings, StackedEmbeddings, CharacterEmbeddings, MuseCrosslingualEmbeddings, ELMoEmbeddings
from typing import List

from flair.embeddings import FlairEmbeddings, TransformerWordEmbeddings,TokenEmbeddings

word_embedding = MuseCrosslingualEmbeddings()

bert_embedding = TransformerWordEmbeddings('bert-base-multilingual-cased')
embeddings_types= StackedEmbeddings(embeddings=[CharacterEmbeddings(), bert_embedding])

In [ ]:
from flair.models import SequenceTagger
tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings_types,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)
# from flair.models import SequenceTagger

# tagger = SequenceTagger.load('ner')
print(tagger)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(tagger))

SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias

In [ ]:
from flair.trainers import ModelTrainer

trainer : ModelTrainer = ModelTrainer(tagger, corpus)

trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              # train_with_dev=True,
              # monitor_test=True,
              max_epochs=50)

2021-03-11 12:29:40,118 ----------------------------------------------------------------------------------------------------
2021-03-11 12:29:40,122 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): MuseCrosslingualEmbeddings()
    (list_embedding_1): CharacterEmbeddings(
      (char_embedding): Embedding(275, 25)
      (char_rnn): LSTM(25, 25, bidirectional=True)
    )
    (list_embedding_2): TransformerWordEmbeddings(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(119547, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAtt

/usr/local/lib/python3.7/dist-packages/flair/embeddings/token.py:1580: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  word_embedding, device=flair.device, dtype=torch.float


2021-03-11 12:30:03,707 Loading up MUSE embeddings for 'hr'!
2021-03-11 12:30:16,360 epoch 1 - iter 16/161 - loss 14.76506644 - samples/sec: 14.14 - lr: 0.100000
2021-03-11 12:30:30,536 epoch 1 - iter 32/161 - loss 10.68159711 - samples/sec: 36.12 - lr: 0.100000
2021-03-11 12:30:30,683 Loading up MUSE embeddings for 'no'!
2021-03-11 12:30:51,539 epoch 1 - iter 48/161 - loss 8.32348785 - samples/sec: 24.38 - lr: 0.100000
2021-03-11 12:30:55,970 Loading up MUSE embeddings for 'ro'!
2021-03-11 12:31:10,547 Loading up MUSE embeddings for 'ca'!
2021-03-11 12:31:18,393 Loading up MUSE embeddings for 'fr'!
2021-03-11 12:31:27,652 epoch 1 - iter 64/161 - loss 7.00176718 - samples/sec: 14.18 - lr: 0.100000
2021-03-11 12:31:40,685 Loading up MUSE embeddings for 'id'!
2021-03-11 12:31:41,165 https://flair.informatik.hu-berlin.de/resources/embeddings/muse/muse.id.vec.gensim.vectors.npy not found in cache, downloading to /tmp/tmp0hjann21


100%|██████████| 240000080/240000080 [00:14<00:00, 16272247.64B/s]

2021-03-11 12:31:56,420 copying /tmp/tmp0hjann21 to cache at /root/.flair/embeddings/MUSE/muse.id.vec.gensim.vectors.npy


2021-03-11 12:31:58,517 removing temp file /tmp/tmp0hjann21
2021-03-11 12:32:02,791 https://flair.informatik.hu-berlin.de/resources/embeddings/muse/muse.id.vec.gensim not found in cache, downloading to /tmp/tmpocm6itxs


100%|██████████| 10614113/10614113 [00:01<00:00, 7107204.24B/s]

2021-03-11 12:32:04,771 copying /tmp/tmpocm6itxs to cache at /root/.flair/embeddings/MUSE/muse.id.vec.gensim


2021-03-11 12:32:04,784 removing temp file /tmp/tmpocm6itxs
2021-03-11 12:32:06,248 epoch 1 - iter 80/161 - loss 6.08155067 - samples/sec: 13.27 - lr: 0.100000
2021-03-11 12:32:17,318 Loading up MUSE embeddings for 'da'!
2021-03-11 12:32:28,550 epoch 1 - iter 96/161 - loss 5.62066316 - samples/sec: 22.96 - lr: 0.100000
2021-03-11 12:32:30,828 Loading up MUSE embeddings for 'fi'!
2021-03-11 12:32:47,814 Loading up MUSE embeddings for 'et'!
2021-03-11 12:32:55,671 Loading up MUSE embeddings for 'it'!
2021-03-11 12:33:05,318 epoch 1 - iter 112/161 - loss 5.01753703 - samples/sec: 13.93 - lr: 0.100000
2021-03-11 12:33:16,400 Loading up MUSE embeddings for 'nl'!
2021-03-11 12:33:26,917 epoch 1 - iter 128/161 - loss 4.45110999 - samples/sec: 23.71 - lr: 0.100000
2021-03-11 12:33:41,454 epoch 1 - iter 144/161 - loss 3.98645792 - samples/sec: 35.23 - lr: 0.100000
2021-03-11 12:33:56,043 epoch 1 - iter 160/161 - loss 3.61502726 - samples/sec: 35.10 - lr: 0.100000
2021-03-11 12:33:56,331 -------

{'dev_loss_history': [2.9712154865264893,
  0.9905304312705994,
  0.8376005291938782,
  1.0136915445327759,
  0.7377455830574036,
  0.7242583632469177,
  0.6750032305717468,
  0.6758297085762024,
  0.6810588836669922,
  0.6823708415031433,
  0.8654462099075317,
  0.6553251147270203,
  0.7124163508415222,
  0.6431124806404114,
  0.6884129047393799,
  0.7235537767410278,
  0.6823051571846008,
  0.6383929252624512,
  0.6734601855278015,
  0.6518872976303101,
  0.6605895161628723,
  0.6677911877632141,
  0.6627979874610901,
  0.6384405493736267,
  0.6412671804428101,
  0.6555718779563904,
  0.6319429278373718,
  0.6334441304206848,
  0.6346413493156433,
  0.6459820866584778,
  0.6542726159095764,
  0.6493107676506042,
  0.6523300409317017,
  0.6480966806411743,
  0.652953565120697,
  0.6584500670433044,
  0.6537882685661316,
  0.6528280377388,
  0.6529477834701538,
  0.6533915400505066,
  0.6528453230857849,
  0.6524314284324646,
  0.6522425413131714,
  0.6535018682479858,
  0.653716385364

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger# load the trained model

model = SequenceTagger.load('/content/resources/taggers/example-ner/best-model.pt')# create example sentence
sentence = Sentence("Olen Helsingissä")# predict the tags
sentence2 = Sentence("running in Seurasaari")
model.predict(sentence)
model.predict(sentence2)
print(sentence.to_tagged_string())
print(sentence2.to_tagged_string())

2021-03-11 13:57:19,842 loading file /content/resources/taggers/example-ner/best-model.pt
Olen Helsingissä <B-location>
running in Seurasaari <B-location>


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!zip -r /content/resources.zip /content/resources/

In [ ]:
!cp resources.zip '/content/drive/MyDrive/Trained_model_geoparser/multilingual/'
!ls -lt '/content/drive/MyDrive/Trained_model_geoparser/multilingual/' 